In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [83]:
df = pd.read_csv("../datasets/train_radiomics_hipocamp.csv")

In [40]:
df = df.loc[df['Transition'] != 'CN-MCI']

In [60]:
df['Transition'].value_counts()

Transition
CN-CN      96
MCI-MCI    71
MCI-AD     68
AD-AD      60
CN-MCI     10
Name: count, dtype: int64

In [26]:
df['Transition'] = df['Transition'].map({
    'CN-CN': 0,
    'CN-MCI': 1,
    'MCI-MCI': 2,
    'MCI-AD': 3,
    'AD-AD': 4
})

In [84]:
X = df.drop(['Transition'], axis=1)     
y = df['Transition'].to_frame()         

In [85]:
df_numerico = X.select_dtypes(include=[np.number])

In [73]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = MinMaxScaler()
df_numerico = pd.DataFrame(scaler.fit_transform(df_numerico), columns=df_numerico.columns, index=df_numerico.index) #normalização

In [5]:
from sklearn.ensemble import RandomForestClassifier

# Treinar um modelo Random Forest
model = RandomForestClassifier(random_state=2021)
model.fit(df_numerico, y)

# Obter as importâncias das features
feature_importances = pd.DataFrame({
    'Feature': df_numerico.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)

# Mostrar as 10 features mais importantes
#print(feature_importances.head(10))

# Selecionar as n melhores features
n = 500
features_to_keep = feature_importances.iloc[:n]['Feature']

# Filtrar o DataFrame original para manter somente essas features
df_numerico = df_numerico[features_to_keep]

c:\Users\super\anaconda3\envs\myenv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [6]:
from sklearn.linear_model import LogisticRegression
logmodel1 = LogisticRegression(random_state=2022, solver='newton-cg')
logmodel1.fit(df_numerico, y)

c:\Users\super\anaconda3\envs\myenv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=2022, solver='newton-cg')

In [86]:
X_test = pd.read_csv("../datasets/test_radiomics_hipocamp.csv")
X_test = X_test.select_dtypes(include=[np.number])
#X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns, index=X_test.index)
#X_test = X_test[features_to_keep]

In [44]:
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=2022)
model.fit(df_numerico, y)

ValueError: could not convert string to float: 'CN-CN'

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='rbf', C=100, random_state=2022)
model.fit(df_numerico, y)

c:\Users\super\anaconda3\envs\myenv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1, random_state=2022)

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(df_numerico, y)


c:\Users\super\anaconda3\envs\myenv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [88]:
predictions = model.predict(X_test)
predictions

array(['MCI-AD', 'CN-MCI', 'CN-MCI', 'CN-CN', 'CN-MCI', 'MCI-MCI',
       'AD-AD', 'CN-CN', 'MCI-MCI', 'AD-AD', 'MCI-AD', 'CN-MCI', 'MCI-AD',
       'CN-MCI', 'AD-AD', 'AD-AD', 'CN-MCI', 'MCI-AD', 'MCI-MCI', 'AD-AD',
       'AD-AD', 'MCI-AD', 'MCI-MCI', 'AD-AD', 'AD-AD', 'MCI-AD', 'AD-AD',
       'AD-AD', 'CN-CN', 'MCI-MCI', 'CN-CN', 'MCI-MCI', 'MCI-AD', 'AD-AD',
       'MCI-MCI', 'MCI-AD', 'AD-AD', 'CN-CN', 'CN-MCI', 'CN-CN', 'MCI-AD',
       'CN-CN', 'AD-AD', 'CN-MCI', 'CN-CN', 'MCI-AD', 'MCI-MCI', 'CN-MCI',
       'MCI-MCI', 'MCI-MCI', 'MCI-AD', 'MCI-MCI', 'MCI-AD', 'AD-AD',
       'MCI-MCI', 'AD-AD', 'CN-CN', 'CN-MCI', 'AD-AD', 'MCI-AD', 'CN-CN',
       'AD-AD', 'CN-MCI', 'AD-AD', 'MCI-MCI', 'MCI-MCI', 'CN-CN',
       'CN-MCI', 'MCI-AD', 'CN-CN', 'MCI-MCI', 'AD-AD', 'CN-CN', 'AD-AD',
       'AD-AD', 'AD-AD', 'MCI-AD', 'MCI-AD', 'CN-CN', 'CN-CN', 'CN-CN',
       'MCI-AD', 'AD-AD', 'MCI-MCI', 'AD-AD', 'MCI-AD', 'AD-AD', 'AD-AD',
       'CN-CN', 'CN-MCI', 'AD-AD', 'MCI-MCI', 'MCI-AD',

In [32]:
predictions = [round(pred) for pred in predictions]

reverse_mapping = {
    0: 'CN-CN',
    1: 'CN-MCI',
    2: 'MCI-MCI',
    3: 'MCI-AD',
    4: 'AD-AD'
}

predictions = [reverse_mapping[pred] for pred in predictions]
predictions

['MCI-MCI',
 'MCI-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'MCI-MCI',
 'AD-AD',
 'CN-MCI',
 'MCI-AD',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'MCI-MCI',
 'MCI-AD',
 'MCI-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'MCI-AD',
 'MCI-MCI',
 'MCI-AD',
 'MCI-AD',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'MCI-MCI',
 'MCI-AD',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'MCI-AD',
 'CN-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-AD',
 'CN-MCI',
 'MCI-MCI',
 'MCI-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'MCI-AD',
 'CN-MCI',
 'AD-AD',
 'CN-CN',
 'CN-MCI',
 'MCI-AD',
 'MCI-MCI',
 'CN-MCI',
 'MCI-AD',
 'MCI-MCI',
 'MCI-AD',
 'MCI-MCI',
 'MCI-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'CN-MCI',
 'MCI-MCI',
 'MCI-AD',
 'CN-MCI',
 'MCI-MCI',
 'MCI-AD',
 'MCI-AD',
 'MCI-MCI',
 'MCI-MCI',
 'MCI-MCI',
 'CN-MCI',
 'CN-MCI',
 'MCI-MCI',
 'MCI-AD',
 'MCI-MCI',
 'MCI-AD',
 'MCI-MCI',
 'MCI-AD',
 'M

In [89]:
data = pd.DataFrame({
    'RowId': np.arange(1, len(predictions) + 1), 
    'Result': predictions
})

data.to_csv('naiveBaysNaoNormalizado.csv', index=False)